In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Model download from Hugging Face

In [1]:
# @title Install Vertex AI SDK and other required packages
%pip install --upgrade --user --quiet google-cloud-aiplatform \
                                      huggingface_hub[hf_transfer] \
                                      transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 59.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# @title Define constants
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
BUCKET_URI = "gs://sllm_checkpoints"  # @param {type:"string"}
MODEL_DIR = "EleutherAI/gpt-j-6b"  # @param {type:"string"}


In [3]:
# @title GCP Authentication

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)


In [4]:
# @title Authenticate your Hugging Face account
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) y


## Download a model using command

In [5]:
# @title Create local dir.
LOCAL_DIR=MODEL_DIR
! mkdir -p {LOCAL_DIR}

In [6]:
# @title Create a bucket.
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://sllm_checkpoints/...
ServiceException: 409 A Cloud Storage bucket named 'sllm_checkpoints' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [7]:
# @title Download model from Hugging face

#MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
#MODEL_NAME = "microsoft/Phi-3.5-MoE-instruct"
#MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"
MODEL_NAME = "EleutherAI/gpt-j-6b"

! huggingface-cli download {MODEL_NAME} --exclude "*.bin" "*.pth" "*.gguf" ".gitattributes" --local-dir {LOCAL_DIR}

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
tf_model.h5:  41% 9.85G/24.2G [01:46<02:20, 102MB/s] 


flax_model.msgpack:  40% 9.66G/24.2G [01:46<02:07, 114MB/s]



tf_model.h5:  41% 9.86G/24.2G [01:46<02:26, 97.9MB/s]


flax_model.msgpack:  40% 9.68G/24.2G [01:46<02:03, 118MB/s]



tf_model.h5:  41% 9.88G/24.2G [01:46<02:23, 100MB/s] 



tf_model.h5:  41% 9.89G/24.2G [01:47<02:24, 99.2MB/s]


flax_model.msgpack:  40% 9.70G/24.2G [01:47<02:18, 105MB/s]



tf_model.h5:  41% 9.90G/24.2G [01:47<02:22, 100MB/s] 



tf_model.h5:  41% 9.91G/24.2G [01:47<02:21, 101MB/s]


flax_model.msgpack:  40% 9.72G/24.2G [01:47<02:19, 104MB/s]



tf_model.h5:  41% 9.92G/24.2G [01:47<02:24, 98.9MB/s]


flax_model.msgpack:  40% 9.74G/24.2G [01:47<02:17, 105MB/s]



tf_model.h5:  41% 9.94G/24.2G [01:47<02:09, 110MB/s] 



tf_model.h5:  41% 9.96G/24.2G [01:47<02:02, 117MB/s]


flax_model.msgpack:  40% 9.76G/24.2G [01:47<02:22, 102MB/s]


flax_model.msgpack:  40% 9.77G/24.2G [01:47<02:25, 99.3MB/s]



tf_model.h5:  41% 

In [8]:
# @title Upload model files into GCS
! gsutil -o GSUtil:parallel_composite_upload_threshold=150M -m cp -e -r {LOCAL_DIR}/* {BUCKET_URI}/{MODEL_DIR}

Copying file://EleutherAI/gpt-j-6b/added_tokens.json [Content-Type=application/json]...
Copying file://EleutherAI/gpt-j-6b/flax_model.msgpack [Content-Type=application/octet-stream]...
Copying file://EleutherAI/gpt-j-6b/config.json [Content-Type=application/json]...
Copying file://EleutherAI/gpt-j-6b/merges.txt [Content-Type=text/plain]...
Copying file://EleutherAI/gpt-j-6b/special_tokens_map.json [Content-Type=application/json]...
Copying file://EleutherAI/gpt-j-6b/tf_model.h5 [Content-Type=application/x-hdf5]...
Copying file://EleutherAI/gpt-j-6b/README.md [Content-Type=text/markdown]...
Copying file://EleutherAI/gpt-j-6b/tokenizer_config.json [Content-Type=application/json]...
Copying file://EleutherAI/gpt-j-6b/vocab.json [Content-Type=application/json]...
Copying file://EleutherAI/gpt-j-6b/tokenizer.json [Content-Type=application/json]...
- [10/10 files][ 45.1 GiB/ 45.1 GiB] 100% Done  47.3 MiB/s ETA 00:00:00         
Operation completed over 10 objects/45.1 GiB.                   

## Download model using python code

In [ ]:
# @title Helper function to download a model and store it into GCS
from transformers import AutoModel, AutoTokenizer
from google.cloud import storage

def model_download(model_name:str,
                   save_path:str,
                   bucket_name:str
                   ):

    # Download model and tokenizer.
    model = AutoModel.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    # Save the model into Cloud storage.
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    for file_name in os.listdir(save_path):
      blob = bucket.blob(f"{save_path}/{file_name}")
      with open(os.path.join(save_path, file_name), "rb") as f:
        blob.upload_from_file(f)

    return model, tokenizer

In [ ]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
save_path = "model"
bucket_name = "sllm_0106"

model, tokenizer = model_download(model_name,save_path,bucket_name)
